In [28]:
import base64
import collections
import datetime
import itertools
import json
import math
import os
import operator
import pickle
import random
import re
import sys
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab
import scipy
import seaborn as sns
import sklearn
from sklearn import *
import statsmodels as sm
from bokeh import plotting, models, layouts, palettes

np.random.seed(1337)

%matplotlib inline

sns.set(font_scale=1.0)
mpl.rcParams['figure.figsize'] = 10, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

plotting.output_notebook()

Loading BokehJS ...

### Using Palettes

In [2]:
from bokeh.palettes import Spectral6
Spectral6

['#3288bd', '#99d594', '#e6f598', '#fee08b', '#fc8d59', '#d53e4f']

In [3]:
print len(palettes.all_palettes)

44


### Line Properties

In [4]:
p = plotting.figure(plot_width=400, plot_height=400)

p.line(
    [1, 2, 3, 4, 5],
    [6, 7, 2, 4, 5],
    line_width=4,
    line_color='red',
    line_join='bevel',
    line_alpha=0.6,
    line_cap='butt',
    line_dash='dotted')

plotting.show(p)

### Fill Properties

In [5]:
p = plotting.figure(plot_width=400, plot_height=400)

p.rect(
    x=[1, 2, 3],
    y=[1, 2, 3],
    width=0.2,
    height=40,
    angle=math.pi/3,
    height_units="screen",
    fill_color='red',
    fill_alpha=0.5)

plotting.show(p)

### Text Properties

In [6]:
p = plotting.figure(plot_width=300, plot_height=300)

p.circle([1,2], [3,4])

p.title.text = "Title With Options"
p.title.align = "right"
p.title.text_color = "blue"
p.title.text_font_size = "25px"
p.title.background_fill_color = "yellow"
p.title.text_font = 'Courier New'

plotting.show(p)

### Visibility

In [7]:
# We set-up a standard figure with two lines
p = plotting.figure(plot_width=500, plot_height=200, tools='')
visible_line = p.line([1, 2, 3], [1, 2, 1], line_color="blue")
invisible_line = p.line([1, 2, 3], [2, 1, 2], line_color="pink")

# We hide the xaxis, the xgrid lines, and the pink line
invisible_line.visible = False
p.xaxis.visible = False
p.xgrid.visible = False

plotting.show(p)

In [8]:
from bokeh.models import Toggle, BoxAnnotation, CustomJS

# We set-up the same standard figure with two lines and now a box over top
p = plotting.figure(plot_width=600, plot_height=200, tools='')
visible_line = p.line([1, 2, 3], [1, 2, 1], line_color="blue")
invisible_line = p.line([1, 2, 3], [2, 1, 2], line_color="pink")

box = BoxAnnotation(left=1.5, right=2.5, fill_color='green', fill_alpha=0.1)
p.add_layout(box)

# We write coffeescript to link toggle with visible property of box and line
code = '''\
object.visible = toggle.active
'''

callback1 = CustomJS.from_coffeescript(code=code, args={})
toggle1 = Toggle(label="Green Box", button_type="success", callback=callback1)
callback1.args = {'toggle': toggle1, 'object': box}

callback2 = CustomJS.from_coffeescript(code=code, args={})
toggle2 = Toggle(label="Pink Line", button_type="success", callback=callback2)
callback2.args = {'toggle': toggle2, 'object': invisible_line}

plotting.show(layouts.layout([p], [toggle1, toggle2]))

### Background

In [9]:
p = plotting.figure(plot_width=400, plot_height=400)
p.background_fill_color = "beige"
p.background_fill_alpha = 0.5

p.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=10)

plotting.show(p)

### Styling Glyphs

In [10]:
p = plotting.figure(plot_width=400, plot_height=400)
r = p.circle([1,2,3,4,5], [2,5,8,2,7])

glyph = r.glyph
glyph.size = 60
glyph.fill_alpha = 0.2
glyph.line_color = "firebrick"
glyph.line_dash = [6, 3]
glyph.line_width = 2

plotting.show(p)

### Selecting Glyphs

In [11]:
plot = plotting.figure(plot_width=400, plot_height=400, tools="tap", title="Select a circle")
renderer = plot.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=50)

selected_circle = models.Circle(fill_alpha=1, fill_color="firebrick", line_color=None)
nonselected_circle = models.Circle(fill_alpha=0.2, fill_color="blue", line_color="firebrick")

renderer.selection_glyph = selected_circle
renderer.nonselection_glyph = nonselected_circle

plotting.show(plot)

In [12]:
plot = plotting.figure(plot_width=400, plot_height=400, tools="tap", title="Select a circle")

renderer = plot.circle(
    [1, 2, 3, 4, 5],
    [2, 5, 8, 2, 7],
    size=50,
    
    # set visual properties for selected glyphs
    selection_color="firebrick",

    # set visual properties for non-selected glyphs
    nonselection_fill_alpha=0.2,
    nonselection_fill_color="blue",
    nonselection_line_color="firebrick",
    nonselection_line_alpha=1.0)

plotting.show(plot)

### Hover

In [13]:
from bokeh.sampledata.glucose import data

subset = data.ix['2010-10-06']

x, y = subset.index.to_series(), subset['glucose']

# Basic plot setup
plot = plotting.figure(plot_width=600, plot_height=300, x_axis_type="datetime", tools="",
              toolbar_location=None, title='Hover over points')

plot.line(x, y, line_dash="4 4", line_width=1, color='gray')

cr = plot.circle(x, y, size=20,
                fill_color="grey", hover_fill_color="firebrick",
                fill_alpha=0.05, hover_alpha=0.3,
                line_color=None, hover_line_color="white")

plot.add_tools(models.HoverTool(tooltips=None, renderers=[cr], mode='hline'))

plotting.show(plot)

### Axes

In [14]:
p = plotting.figure(plot_width=400, plot_height=400)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

# change just some things about the x-axes
p.xaxis.axis_label = "Temp"
p.xaxis.axis_line_width = 3
p.xaxis.axis_line_color = "red"

# change just some things about the y-axes
p.yaxis.axis_label = "Pressure"
p.yaxis.major_label_text_color = "orange"
p.yaxis.major_label_orientation = "vertical"

# change things on all axes
p.axis.minor_tick_in = -3
p.axis.minor_tick_out = 6

plotting.show(p)

### Labels

In [15]:
p = plotting.figure(plot_width=400, plot_height=400)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

p.xaxis.axis_label = "Lot Number"
p.xaxis.axis_label_text_color = "#aa6666"
p.xaxis.axis_label_standoff = 30

p.yaxis.axis_label = "Bin Count"
p.yaxis.axis_label_text_font_style = "italic"

plotting.show(p)

### Bounds

In [16]:
p = plotting.figure(plot_width=400, plot_height=400)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

p.xaxis.bounds = (2, 4)
p.yaxis.bounds = (3, 7)

plotting.show(p)

### Tick Locations

In [17]:
p = plotting.figure(plot_width=400, plot_height=400)
p.circle([1.5,2.2,3.3,4.1,4.9], [2,5,8,2,7], size=10)

p.xaxis.ticker = [1.5,2.2,3.3,4.1,4.9]
p.yaxis.ticker = [2,5,8,2,7]

plotting.show(p)

### Tick Lines

In [18]:
p = plotting.figure(plot_width=400, plot_height=400)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

p.xaxis.major_tick_line_color = "firebrick"
p.xaxis.major_tick_line_width = 3
p.xaxis.minor_tick_line_color = "orange"

p.yaxis.minor_tick_line_color = None

p.axis.major_tick_out = 10
p.axis.minor_tick_in = -3
p.axis.minor_tick_out = 8

plotting.show(p)

### Tick Labels

In [19]:
p = plotting.figure(plot_width=400, plot_height=400)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

p.xaxis[0].formatter = models.PrintfTickFormatter(format="%4.1e")
p.yaxis[0].formatter = models.PrintfTickFormatter(format="%5.3f mu")

plotting.show(p)

In [22]:
p = plotting.figure(plot_width=400, plot_height=400)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

p.xaxis.major_label_orientation = math.pi/4
p.yaxis.major_label_orientation = "vertical"

plotting.show(p)

### Grid Lines

In [23]:
p = plotting.figure(plot_width=400, plot_height=400)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

# change just some things about the x-grid
p.xgrid.grid_line_color = None

# change just some things about the y-grid
p.ygrid.grid_line_alpha = 0.8
p.ygrid.grid_line_dash = [6, 4]

plotting.show(p)

### Minor Grid Lines

In [24]:
p = plotting.figure(plot_width=400, plot_height=400)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

# change just some things about the y-grid
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.1

plotting.show(p)

### Legends

In [25]:
x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)

p = plotting.figure(plot_width=400, plot_height=400)

p.circle(x, y, legend="sin(x)")
p.line(x, y, legend="sin(x)")

p.line(x, 2*y, legend="2*sin(x)",
       line_dash=[4, 4], line_color="orange", line_width=2)

p.square(x, 3*y, legend="3*sin(x)", fill_color=None, line_color="green")
p.line(x, 3*y, legend="3*sin(x)", line_color="green")

p.legend.location = "bottom_left"

plotting.show(p)

In [26]:
x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)

p = plotting.figure(plot_width=400, plot_height=400, toolbar_location="above")

r0 = p.circle(x, y)
r1 = p.line(x, y)

r2 = p.line(x, 2*y, line_dash=[4, 4], line_color="orange", line_width=2)

r3 = p.square(x, 3*y, fill_color=None, line_color="green")
r4 = p.line(x, 3*y, line_color="green")

legend = models.Legend(items=[
    ("sin(x)"   , [r0, r1]),
    ("2*sin(x)" , [r2]),
    ("3*sin(x)" , [r3, r4]),
], location=(0, -30))

p.add_layout(legend, 'right')

plotting.show(p)

In [27]:
x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)

p = plotting.figure(plot_width=400, plot_height=400)

p.circle(x, y, legend="sin(x)")
p.line(x, y, legend="sin(x)")

p.line(x, 2*y, legend="2*sin(x)",
       line_dash=[4, 4], line_color="orange", line_width=2)

p.square(x, 3*y, legend="3*sin(x)", fill_color=None, line_color="green")
p.line(x, 3*y, legend="3*sin(x)", line_color="green")

p.legend.location = "bottom_left"
p.legend.border_line_color = None

plotting.show(p)